# Challenge 1 - Low Conversion

## 1️⃣ Setup
### 🧰 Installing Selling Partner API SDK

The **Amazon Selling Partner API SDK** is an official Amazon library that simplifies integration with Amazon’s SP-API. It handles things like authentication, request signing, and response parsing — so you can focus on building your solution instead of dealing with low-level API mechanics.

With this SDK, you can easily access endpoints like Data Kiosk, Listings, Pricing using clean, structured Python code.

📚 Learn more or explore the source on GitHub: https://github.com/amzn/selling-partner-api-sdk/tree/main


📦 To install the SDK, run the below block 👇

In [1]:
pip install amzn-sp-api==1.2.0


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: /usr/local/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---

### 🐛 Mocking the SDK


Luckily the Amazon Solution Architect team created a mock server that can replicate the API responses without having credentials.

Below you will find `mock_oauth_endpoint` and `mock_endpoint` variables that can be used to skip the credentials.

In [5]:
from spapi.models.datakiosk_v2023_11_15 import GetDocumentResponse, CreateQueryResponse, GetQueriesResponse, Query
from spapi import SPAPIConfig, SPAPIClient, ApiException, QueriesApi, ListingsApi

# Configuration for mock SP-API
mock_oauth_endpoint = "http://localhost:8000/auth/o2/token"
mock_endpoint = "http://localhost:8000"

# Filling the below client_id, client_secret and refresh_token credentials is NOT needed for MOCK
config = SPAPIConfig(
    client_id="MOCK_CLIENT_ID",
    client_secret="MOCK_CLIENT_SECRET",
    refresh_token="MOCK_REFRESH_TOKEN",
    region="NA"
)

# Initializing MOCK SP-API SDK Client
client = SPAPIClient(config, endpoint=mock_endpoint, oauth_endpoint=mock_oauth_endpoint)

data_kiosk_api = QueriesApi(client.api_client)
listings_api = ListingsApi(client.api_client)
print("💻 API Client initiated")

💻 API Client initiated


## 2️⃣ Challenge

### 🕵️‍♀️ Investigate Low Conversion Listings

In recent reporting, we noticed something unusual with our Listings Sales Conversion.

### 📊 Visualize Your Economics Data

🔍 **What is Data Kiosk?**

Data Kiosk lets you submit GraphQL queries from a variety of schemas to help manage selling partner businesses. You can use the following SDK to call Data Kiosk API.

It’s your all-access pass to query Amazon datasets like sales, traffic, and profitability using GraphQL.

You can:

- Submit custom queries
- Check their status
- Download results
  
🙌 Time to get seller data with the power of GraphQL!

In [ ]:
pip install pandas

In [6]:
graphql_query_str = ("query { analytics_economics_2024_03_15 { economics( marketplaceIds: [\"ATVPDKIKX0DER\"], "
                 "startDate: \"2025-06-01\", endDate: \"2025-06-05\", "
                 "aggregateBy: { date: DAY, productId: MSKU } ) "
                 "{ startDate endDate msku marketplaceId sales { unitsOrdered netUnitsSold "
                 "netProductSales { amount currencyCode } } netProceeds { total { amount currencyCode } } } } }")
graphql_query = {"query": graphql_query_str}
print("📃 Query: \n", graphql_query_str)

# Tell Amazon to start processing your query
create_query_response = data_kiosk_api.create_query(body=graphql_query)
print("✅ Query submitted! Response:", create_query_response)

# Check the status of your query
get_query_response = data_kiosk_api.get_query(query_id=create_query_response.query_id)
print("⏱️ Query status:", get_query_response)

# Now let's get the actual data!
get_document = data_kiosk_api.get_document(document_id=get_query_response.data_document_id)
print("📊 Here's your data:", get_document)

import ast
import pandas as pd
import matplotlib.pyplot as plt

# Safely convert the string to a dictionary
parsed_document = ast.literal_eval(get_document.document_url)

# Extract the economics data
economics_data = parsed_document["data"]["analytics_economics_2024_03_15"]["economics"]

# Build a DataFrame
df = pd.DataFrame([
    {
        "MSKU": item["msku"],
        "Start Date": item["startDate"],
        "End Date": item["endDate"],
        "Net Product Sales ($)": item["sales"]["netProductSales"]["amount"],
        "Units Sold": item["sales"]["netUnitsSold"]
    }
    for item in economics_data
])

# Show the raw data
print("🧾 Parsed Economics Data:")
display(df)

# Plotting
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar chart for Net Product Sales
ax1.bar(df["MSKU"], df["Net Product Sales ($)"], label='Net Product Sales ($)')
ax1.set_ylabel("Sales ($)", fontsize=12)
ax1.set_title("Net Product Sales vs Units Sold by MSKU", fontsize=14)
ax1.set_xlabel("MSKU", fontsize=12)
ax1.legend(loc="upper left")

# Line chart for Units Sold
ax2 = ax1.twinx()
ax2.plot(df["MSKU"], df["Units Sold"], color='orange', marker='o', label='Units Sold')
ax2.set_ylabel("Units Sold", fontsize=12)
ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()


📃 Query: 
 query { analytics_economics_2024_03_15 { economics( marketplaceIds: ["ATVPDKIKX0DER"], startDate: "2025-06-01", endDate: "2025-06-05", aggregateBy: { date: DAY, productId: MSKU } ) { startDate endDate msku marketplaceId sales { unitsOrdered netUnitsSold netProductSales { amount currencyCode } } netProceeds { total { amount currencyCode } } } } }
✅ Query submitted! Response: {'query_id': '1000001'}
⏱️ Query status: {'created_time': datetime.datetime(2025, 7, 17, 11, 25, 13, 645000, tzinfo=tzutc()),
 'data_document_id': 'usecase_1_document_id.amzn.100001',
 'error_document_id': None,
 'pagination': None,
 'processing_end_time': datetime.datetime(2025, 7, 17, 11, 25, 13, 645000, tzinfo=tzutc()),
 'processing_start_time': datetime.datetime(2025, 7, 17, 11, 25, 13, 645000, tzinfo=tzutc()),
 'processing_status': 'DONE',
 'query': 'query { analytics_economics_2024_03_15 { economics( marketplaceIds: '
          '["ATVPDKIKX0DER"], startDate: "2025-06-01", endDate: "2025-06-05", '
  

ModuleNotFoundError: No module named 'pandas'

---

## 3️⃣ Troubleshooting: 

### 🫆 Clue

We noticed that similar to `MOTOR-GEAR-US`, all the SKUs with the same product type are experiencing low sales in the US marketplace! 

🧐 Interestingly, these same SKUs are performing quite well in Canada and Mexico, which suggests there may be an issue specific to the US marketplace. 

### 🔍 Next Step: Investigate the Listing

* 🔔 Best pratices? 
Subscribe to Notifications - use [LISTINGS_ITEM_ISSUES_CHANGE](https://developer-docs.amazon.com/sp-api/docs/notification-type-values#listings_item_issues_change) and [LISTINGS_ITEM_STATUS_CHANGE](https://developer-docs.amazon.com/sp-api/docs/notification-type-values#listings_item_status_change) notification to get near realtime updates for issues or status changes.

* 💡 Or use [getListingsItems](https://developer-docs.amazon.com/sp-api/reference/getlistingsitem) operation. Execute the code below to call getListingsItems operation on one of the SKUs with the `POWERSPORTS_PROTECTIVE_GEAR` product type in the US to investigate further. 

In [ ]:
import json
get_listing = listings_api.get_listings_item('AMY7FKRUBY7XV', 
                                             'MOTOR-GEAR-US',
                                             ['ATVPDKIKX0DER'], 
                                             included_data=['issues'])

print("🛍️ Here's your listing: \n", json.dumps(get_listing.to_dict(), indent=4, default=str))

## 4️⃣ Resolution: 

### 🧐 Analysis


There is an `ERROR` severity issue in `getListingsItems` response, making it no longer discoverable in the US marketplace! And attribute is missing from the attributes data. 

But here’s the twist 🔀:
* `90220` error code usually means a required attribute is missing.
* And `MISSING_ATTRIBUTE` is a synchronous error—it should have shown up during submission. How come my team did not receive any error when they submitted the listing. What went wrong!!? 🤔

<details>
<summary><strong>Click to see issue</strong></summary>

```
[ {
      "code" : "90220",
      "message" : "Department Name is required but was not provided.",
      "severity" : "ERROR",
      "attributeNames" : [ "department" ],
      "categories" : [ "MISSING_ATTRIBUTE" ],
      "enforcements" : {
        "actions" : [ {
          "action" : "SEARCH_SUPPRESSED"
        } ],
        "exemption" : {
          "status" : "NOT_EXEMPT"
        }
      }
    } ]
```

</details>
<br />

### 🛠️ Fix

Our team pulled the historical `putListingsItem` submissions that created this listing in US and MX marketplace.

<details>
<summary><strong>Click to see MX Marketplace Submission</strong></summary>
    
```
{
    "productType": "POWERSPORTS_PROTECTIVE_GEAR",
    "attributes": {
        "color": [
            {
                "value": "A set-Black",
                "language_tag": "es_MX",
                "marketplace_id": "A1AM78C64UM0Y8"
            }
        ]
        ...
    },
    "requirements": "LISTING"
}
```
</details>

<details>
<summary><strong>Click to see US Marketplace Submission</strong></summary>
    
```
{
    "productType": "POWERSPORTS_VEHICLE_PART",
    "attributes": {
        "color": [
            {
                "value": "A set-Black",
                "language_tag": "en_US",
                "marketplace_id": "ATVPDKIKX0DER"
            }
        ]
        ...
    },
    "requirements": "LISTING"
}
```
</details>
<br />
Can you spot the difference here? 👀


Our team has prepared a patchListingsItem request below. Fill out the correct attributes to update to fix the Listings Error!

<details>
<summary><strong>💡 Click to see HINT 1</strong></summary>
    
_*Products should maintain the same product type across all marketplaces. While it's technically possible to assign different product types in different marketplaces, this practice often leads to complications. The catalog system may automatically reclassify your contributions, resulting in missing required attributes, incorrect variation themes, and other technical issues.*_

</details>
<details>
<summary><strong>💡 Click to see HINT 2</strong></summary>
    
_*Check getListingsItem response `summary` object to find the correct product type to use*_

</details>
<details>
<summary><strong>💡 Click to see HINT 3</strong></summary>
    
_*Path value typically follows this format: `/attributes/attribute_name`*_

</details>


In [ ]:
import json
body = {
  "productType": # provide value,
  "patches": [
    {
      "op": "replace",
      "path": # provide value,
      "value": [
        {
          "language_tag": "en_US",
          "marketplace_id": 1,
          "value": "Unisex"
        }
      ]
    }
  ]
} 
patch_listing = listings_api.patch_listings_item('AMY7FKRUBY7XV', 'MOTOR-GEAR-US',['ATVPDKIKX0DER'], body)
print("💾 Submission result: \n", json.dumps(patch_listing.to_dict(), indent=4, default=str))